In [57]:
import sys
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [58]:
df = pd.read_csv("loan.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
def getClass(status):
    if status=="Charged Off" or status=="Default" or status=="Does not meet the credit policy. Status:Charged Off" or status=="In Grace Period" or status=="Late (16-30 days)" or status=="Late (31-120 days)":
        return 0
    else:
        return 1
df["goodLoan"] = df["loan_status"].apply(lambda x: getClass(x))

In [60]:
midWest = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
northEast = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']
southEast = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
southWest = ['AZ', 'TX', 'NM', 'OK']
df['region'] = np.nan
def getRegion(state):
    if state in west:
        return 'West'
    elif state in southWest:
        return 'SouthWest'
    elif state in southEast:
        return 'SouthEast'
    elif state in midWest:
        return 'MidWest'
    elif state in northEast:
        return 'NorthEast'
df['region'] = df['addr_state'].apply(lambda x: getRegion(x))

In [61]:
def getIncomeCat(income):
    if income<50000:
        return "Low"
    elif income>=50000 and income<100000:
        return "Middle"
    else: 
        return "High"
df["incomeCat"] = df["annual_inc"].apply(lambda x: getIncomeCat(x))

In [62]:
finaldf = df.copy()
"""
pub_rec - # of accounts client is delinquent
total_acc - # of credit lines the borrower has
delinq_2yrs - # of delinquencies in past 2 yrs
"""
finaldf = finaldf[['loan_amnt', "region", "incomeCat", 'term', 'int_rate', 'installment', 'grade',
       'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan',
       'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
       'goodLoan', 'mths_since_last_major_derog', 'collection_recovery_fee', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
        'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
        'last_pymnt_amnt']]

In [63]:
df_subset = finaldf.sample(n=10000)

In [64]:
df_subset.select_dtypes(include=['object']).head()

,region,incomeCat,term,grade,home_ownership,verification_status,pymnt_plan,purpose
231419,West,Low,36 months,C,RENT,Not Verified,n,debt_consolidation
303701,SouthWest,Low,36 months,D,RENT,Verified,n,debt_consolidation
18066,SouthEast,High,60 months,B,MORTGAGE,Source Verified,n,credit_card
259879,NorthEast,Low,60 months,C,OWN,Source Verified,n,debt_consolidation
2105710,NorthEast,Low,60 months,E,RENT,Verified,n,debt_consolidation


In [65]:
col_object = df_subset.select_dtypes(include=['object']).columns.values.tolist()
col_object

['region',
 'incomeCat',
 'term',
 'grade',
 'home_ownership',
 'verification_status',
 'pymnt_plan',
 'purpose']

In [66]:
expDf = pd.get_dummies(data = df_subset, columns = col_object)

In [67]:
expDf.shape

(10000, 66)

In [69]:
expCol = expDf.columns.values
expCol

array(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'goodLoan',
       'mths_since_last_major_derog', 'collection_recovery_fee',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'last_pymnt_amnt', 'region_MidWest', 'region_NorthEast',
       'region_SouthEast', 'region_SouthWest', 'region_West',
       'incomeCat_High', 'incomeCat_Low', 'incomeCat_Middle',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B',
       'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verificati

In [70]:
expDf = expDf.dropna(axis='columns')
testdf = expDf.copy()

In [71]:
expCol1 = ['loan_amnt', 'int_rate', 'installment', 'annual_inc',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'total_acc', 'collection_recovery_fee',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'last_pymnt_amnt', 'region_MidWest', 'region_NorthEast',
       'region_SouthEast', 'region_SouthWest', 'region_West',
       'incomeCat_High', 'incomeCat_Low', 'incomeCat_Middle',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B',
       'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verification_status_Verified', 'pymnt_plan_n', 'pymnt_plan_y']
for i in range(len(expCol1)):
    for j in range(len(expCol1)):
        testdf['{}_{}'.format(expCol1[i], expCol1[j])] = testdf[expCol1[i]]*testdf[expCol1[j]]
testdf.head()

,loan_amnt,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,...,pymnt_plan_y_grade_G,pymnt_plan_y_home_ownership_ANY,pymnt_plan_y_home_ownership_MORTGAGE,pymnt_plan_y_home_ownership_OWN,pymnt_plan_y_home_ownership_RENT,pymnt_plan_y_verification_status_Not Verified,pymnt_plan_y_verification_status_Source Verified,pymnt_plan_y_verification_status_Verified,pymnt_plan_y_pymnt_plan_n,pymnt_plan_y_pymnt_plan_y
231419,5000,14.47,172.04,42240.0,0.0,0.0,7.0,0.0,4537,7.0,...,0,0,0,0,0,0,0,0,0,0
303701,8000,20.39,298.91,33000.0,0.0,1.0,13.0,0.0,4571,24.0,...,0,0,0,0,0,0,0,0,0,0
18066,35000,11.80,775.03,162000.0,0.0,1.0,10.0,0.0,47120,19.0,...,0,0,0,0,0,0,0,0,0,0
259879,11000,16.46,270.20,45000.0,0.0,0.0,11.0,0.0,6262,15.0,...,0,0,0,0,0,0,0,0,0,0
2105710,13200,19.91,349.06,45000.0,0.0,0.0,6.0,0.0,12951,19.0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
expDf = testdf.copy()
expDf.shape

(10000, 2086)

In [73]:
expDf.columns.tolist()

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'goodLoan',
 'collection_recovery_fee',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'last_pymnt_amnt',
 'region_MidWest',
 'region_NorthEast',
 'region_SouthEast',
 'region_SouthWest',
 'region_West',
 'incomeCat_High',
 'incomeCat_Low',
 'incomeCat_Middle',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'verification_status_Not Verified',
 'verification_status_Source Verified',
 'verification_status_Verified',
 'pymnt_plan_n',
 'pymnt_plan_y',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',

In [74]:
expDf=expDf.drop('goodLoan', axis=1)

In [75]:
purposeList = ['purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding']
purposeDict = dict(zip(range(14), purposeList))
len(purposeList)
purposeDict

{0: 'purpose_car',
 1: 'purpose_credit_card',
 2: 'purpose_debt_consolidation',
 3: 'purpose_educational',
 4: 'purpose_home_improvement',
 5: 'purpose_house',
 6: 'purpose_major_purchase',
 7: 'purpose_medical',
 8: 'purpose_moving',
 9: 'purpose_other',
 10: 'purpose_renewable_energy',
 11: 'purpose_small_business',
 12: 'purpose_vacation',
 13: 'purpose_wedding'}

In [79]:
dfPurpose = []
dfCar = expDf[expDf['purpose_car']==1].reset_index(drop=True)
dfCreditCard = expDf[expDf['purpose_credit_card']==1].reset_index(drop=True)
dfDebtCon = expDf[expDf['purpose_debt_consolidation']==1].reset_index(drop=True)
dfEdu = expDf[expDf['purpose_educational']==1].reset_index(drop=True)
dfHomeImprov = expDf[expDf['purpose_home_improvement']==1].reset_index(drop=True)
dfHouse = expDf[expDf['purpose_house']==1].reset_index(drop=True)
dfMajorPurch = expDf[expDf['purpose_major_purchase']==1].reset_index(drop=True)
dfMed = expDf[expDf['purpose_medical']==1].reset_index(drop=True)
dfMoving = expDf[expDf['purpose_moving']==1].reset_index(drop=True)
dfOther = expDf[expDf['purpose_other']==1].reset_index(drop=True)
dfRenew = expDf[expDf['purpose_renewable_energy']==1].reset_index(drop=True)
dfSmall = expDf[expDf['purpose_small_business']==1].reset_index(drop=True)
dfVaca = expDf[expDf['purpose_vacation']==1].reset_index(drop=True)
dfWed = expDf[expDf['purpose_wedding']==1].reset_index(drop=True)
dfPurpose = [dfCar, dfCreditCard, dfDebtCon, dfEdu, dfHomeImprov, dfHouse, dfMajorPurch, dfMed, dfMoving, 
             dfOther, dfRenew, dfSmall, dfVaca, dfWed]
dfw = [dfCar.copy(), dfCreditCard.copy(), dfDebtCon.copy(), dfEdu.copy(), dfHomeImprov.copy(), dfHouse.copy(), 
       dfMajorPurch.copy(), dfMed.copy(), dfMoving.copy(), dfOther.copy(), dfRenew.copy(), dfSmall.copy(), 
       dfVaca.copy(), dfWed.copy()]

In [80]:
expDf.shape

(10000, 2085)

In [81]:
dfwCar = dfw[0].as_matrix()
qList = []
for row in dfwCar:
    q = np.linalg.norm(row, ord=2)
    qList.append(q)
qList = [q/sum(qList) for q in qList]
x = np.random.choice(range(dfwCar.shape[0]), 1, p=qList)[0]
# S = np.append(S,[dfwCar[x]], axis=0)
x_tilda = dfwCar[x]
dfwCar[0] = dfwCar[0] - (np.dot(dfwCar[0],x_tilda))/ ((np.linalg.norm(x_tilda, ord=2))**2)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [82]:
# dfw.as_matrix()[0] -= np.dot(dfw.as_matrix()[0], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)

In [83]:
dfw = expDf.reset_index(drop=True)
x_tilda
dfw.loc[0] - np.dot(dfw.loc[0], x_tilda)/(np.linalg.norm(x_tilda, ord=2)**2)
# for i in range(dfw.shape[0]):
# dfw.loc[0] -= np.dot(dfw.loc[0], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)

loan_amnt                                            4999.530428
int_rate                                               14.000428
installment                                           171.570428
annual_inc                                          42239.530428
delinq_2yrs                                            -0.469572
inq_last_6mths                                         -0.469572
open_acc                                                6.530428
pub_rec                                                -0.469572
revol_bal                                            4536.530428
total_acc                                               6.530428
collection_recovery_fee                                -0.469572
out_prncp                                            4188.750428
out_prncp_inv                                        4188.750428
total_pymnt                                          1180.420428
total_pymnt_inv                                      1180.420428
total_rec_int            

In [87]:
k = 2030
S = np.empty([0, expDf.shape[1]])
dfS = pd.DataFrame(columns = expDf.columns.values.tolist())
kList = [int(k/14)]*14
dfw = expDf.reset_index(drop=True)
X = dfw.as_matrix()
dfPurpose = [dfCar, dfCreditCard, dfDebtCon, dfEdu, dfHomeImprov, dfHouse, dfMajorPurch, dfMed, dfMoving, 
             dfOther, dfRenew, dfSmall, dfVaca, dfWed]
dfw = [dfCar.copy(), dfCreditCard.copy(), dfDebtCon.copy(), dfEdu.copy(), dfHomeImprov.copy(), dfHouse.copy(), 
       dfMajorPurch.copy(), dfMed.copy(), dfMoving.copy(), dfOther.copy(), dfRenew.copy(), dfSmall.copy(), 
       dfVaca.copy(), dfWed.copy()]
countCar, countCreditCard, countDebtCon, countEdu, countHomeImprov, countHouse, countMajorPurch = 0, 0, 0, 0, 0, 0, 0
countMed, countMoving, countOther, countRenew, countSmall, countVaca, countWed = 0, 0, 0, 0, 0, 0, 0
countList = [countCar, countCreditCard, countDebtCon, countEdu, countHomeImprov, countHouse, countMajorPurch, 
            countMed, countMoving, countOther, countRenew, countSmall, countVaca, countWed]
len(dfS)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


0

In [88]:
t = 0
dfS = pd.DataFrame(columns = expDf.columns.values.tolist())
while dfS.shape[0] < k:
    t += 1
    print("interation (t): {}".format(t))
    i = np.random.choice(14,1)[0]
    X_chosen = dfPurpose[i]
    X_name = purposeDict[i]
    print("Chosen class is: {}".format(X_name))
    X_i = X_chosen.shape[0]
    X_w_chosen = dfw[i].as_matrix()
#     print(X_w_chosen)
    for name in purposeList:
        if name == X_name:
            countList[i] += 1
    if countList[i] < kList[0]:
        qList = []
        for row in X_w_chosen:
            q = np.linalg.norm(row, ord=2)**2
            qList.append(q)
        qList = [q/sum(qList) for q in qList]
        x_i = np.random.choice(range(X_i), 1, p=qList)[0]
        print("Chosen random sample is No.{}".format(x_i))
        x_tilda = X_chosen.as_matrix()[x_i]
        dfS.loc[len(dfS)] = x_tilda   
        for p in range(14):
            X_w_update = dfw[p].as_matrix()
            for i in range(X_w_update.shape[0]):
                X_w_update[i] -= np.dot(X_w_update[i], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)
            dfw[p] = pd.DataFrame(X_w_update, columns = expDf.columns.values.tolist()).reset_index(drop=True)
    else: break
    
print(dfS)

interation (t): 1
Chosen class is: purpose_renewable_energy
[[1.4400e+04 9.4400e+00 3.0201e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.1000e+03 2.2990e+01 1.1999e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [4.0000e+03 1.4460e+01 1.3761e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [2.0000e+03 2.0990e+01 7.5340e+01 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [6.0000e+03 1.4490e+01 2.0650e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.6000e+04 9.4400e+00 3.3557e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]]
Chosen random sample is No.4


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


interation (t): 2
Chosen class is: purpose_credit_card
[[ 3.49973790e+04  9.17903415e+00  7.72409034e+02 ... -2.62096585e+00
  -2.62096585e+00 -2.62096585e+00]
 [ 2.99982646e+03  8.41646307e+00  9.46564631e+01 ... -1.73536927e-01
  -1.73536927e-01 -1.73536927e-01]
 [ 7.39994085e+03  1.26808533e+01  2.48350853e+02 ... -5.91466850e-02
  -5.91466850e-02 -5.91466850e-02]
 ...
 [ 1.99996589e+04  9.73892064e+00  6.45758921e+02 ... -3.41079364e-01
  -3.41079364e-01 -3.41079364e-01]
 [ 1.99996054e+04  1.31653656e+01  4.60425366e+02 ... -3.94634411e-01
  -3.94634411e-01 -3.94634411e-01]
 [ 9.24972258e+03  1.17125814e+01  3.06912581e+02 ... -2.77418558e-01
  -2.77418558e-01 -2.77418558e-01]]
Chosen random sample is No.1412
interation (t): 3
Chosen class is: purpose_moving
[[ 1.29995828e+04  8.74277090e+00  4.13952771e+02 ... -4.17229101e-01
  -4.17229101e-01 -4.17229101e-01]
 [ 7.49952252e+03  1.36025169e+01  2.56152517e+02 ... -4.77483090e-01
  -4.77483090e-01 -4.77483090e-01]
 [ 6.99929744e+03

interation (t): 15
Chosen class is: purpose_wedding
[[ 1.83954849e+04  5.47487254e+00  5.89114873e+02 ... -4.51512746e+00
  -4.51512746e+00 -4.51512746e+00]
 [ 5.99348155e+03  2.42155270e+00  1.84111553e+02 ... -6.51844730e+00
  -6.51844730e+00 -6.51844730e+00]
 [ 9.99483550e+03  1.45554959e+01  3.65055496e+02 ... -5.16450413e+00
  -5.16450413e+00 -5.16450413e+00]
 ...
 [ 3.59659840e+03  1.63183957e+01  1.29878396e+02 ... -3.40160434e+00
  -3.40160434e+00 -3.40160434e+00]
 [ 3.39852583e+03  8.68583419e+00  1.08495834e+02 ... -1.47416581e+00
  -1.47416581e+00 -1.47416581e+00]
 [ 3.99741626e+03  1.61662585e+01  1.43536259e+02 ... -2.58374147e+00
  -2.58374147e+00 -2.58374147e+00]]
Chosen random sample is No.2
interation (t): 16
Chosen class is: purpose_credit_card
[[ 3.49591667e+04 -2.90332528e+01  7.34196747e+02 ... -4.08332528e+01
  -4.08332528e+01 -4.08332528e+01]
 [ 2.99705231e+03  5.64230551e+00  9.18823055e+01 ... -2.94769449e+00
  -2.94769449e+00 -2.94769449e+00]
 [ 7.39871810e+03

interation (t): 27
Chosen class is: purpose_small_business
[[ 1.49909880e+04  1.58379733e+01  5.86197973e+02 ... -9.01202673e+00
  -9.01202673e+00 -9.01202673e+00]
 [ 5.59623946e+03  7.22945651e+00  1.79559457e+02 ... -3.76054349e+00
  -3.76054349e+00 -3.76054349e+00]
 [ 9.93364746e+03 -4.90825442e+01  1.83627456e+02 ... -6.63525442e+01
  -6.63525442e+01 -6.63525442e+01]
 ...
 [ 4.99318067e+03  1.03706668e+01  1.17960667e+02 ... -6.81933319e+00
  -6.81933319e+00 -6.81933319e+00]
 [ 1.59436888e+04  1.95187951e+01  4.69638795e+02 ... -6.31120493e+00
  -6.31120493e+00 -6.31120493e+00]
 [ 1.11923444e+04  8.36439913e+00  2.64834399e+02 ... -7.65560087e+00
  -7.65560087e+00 -7.65560087e+00]]
Chosen random sample is No.106
interation (t): 28
Chosen class is: purpose_house
[[ 4.27321429e+03  2.02042888e+01  1.61464289e+02 ... -1.78571120e+00
  -1.78571120e+00 -1.78571120e+00]
 [ 1.19863009e+04  1.32090837e+00  2.71910908e+02 ... -1.36990916e+01
  -1.36990916e+01 -1.36990916e+01]
 [ 1.39921640e

...

Chosen random sample is No.1475
interation (t): 1759
Chosen class is: purpose_vacation
[[ 2055.01142438 -2925.95857562 -2761.62857562 ... -2944.98857562
  -2944.98857562 -2944.98857562]
 [ 9216.57749733  -770.80250267  -557.82250267 ...  -783.42250267
   -783.42250267  -783.42250267]
 [ 6857.49260055 -5130.97739945 -4746.61739945 ... -5142.50739945
  -5142.50739945 -5142.50739945]
 ...
 [ 3312.53990428  -676.07009572  -555.76009572 ...  -687.46009572
   -687.46009572  -687.46009572]
 [ 5102.17072861 -1385.33927139 -1180.40927139 ... -1397.82927139
  -1397.82927139 -1397.82927139]
 [ 2430.95335115  -759.60664885  -666.62664885 ...  -769.04664885
   -769.04664885  -769.04664885]]
Chosen random sample is No.19
interation (t): 1760
Chosen class is: purpose_moving
[[12064.19537414  -926.64462586  -521.43462586 ...  -935.80462586
   -935.80462586  -935.80462586]
 [ 6584.74793872  -901.17206128  -658.62206128 ...  -915.25206128
   -915.25206128  -915.25206128]
 [ 5616.56186997 -1372.44813003 

interation (t): 1773
Chosen class is: purpose_house
[[ 4093.25619275  -159.75380725   -18.49380725 ...  -181.74380725
   -181.74380725  -181.74380725]
 [10626.20639319 -1358.77360681 -1088.18360681 ... -1373.79360681
  -1373.79360681 -1373.79360681]
 [13186.75731084  -794.99268916  -455.82268916 ...  -813.24268916
   -813.24268916  -813.24268916]
 ...
 [11069.44449295  -913.56550705  -502.77550705 ...  -930.55550705
   -930.55550705  -930.55550705]
 [ 7291.53852545 -1100.62147455  -845.85147455 ... -1108.46147455
  -1108.46147455 -1108.46147455]
 [ 3728.75395626 -1252.25604374 -1087.98604374 ... -1271.24604374
  -1271.24604374 -1271.24604374]]
Chosen random sample is No.21
interation (t): 1774
Chosen class is: purpose_major_purchase
[[ 6.13861268e+02 -3.80028732e+02 -3.55658732e+02 ... -3.86138732e+02
  -3.86138732e+02 -3.86138732e+02]
 [ 2.03411579e+04 -6.45512125e+02 -1.77472125e+02 ... -6.58842125e+02
  -6.58842125e+02 -6.58842125e+02]
 [ 1.70919741e+03 -3.29225894e+01 -5.41258937e+

interation (t): 1786
Chosen class is: purpose_renewable_energy
[[13247.3814538  -1143.1785462   -850.6085462  ... -1152.6185462
  -1152.6185462  -1152.6185462 ]
 [ 2704.19851155  -372.81148845  -275.81148845 ...  -395.80148845
   -395.80148845  -395.80148845]
 [ 2961.09309464 -1024.44690536  -901.29690536 ... -1038.90690536
  -1038.90690536 -1038.90690536]
 ...
 [  411.94767796 -1567.06232204 -1512.71232204 ... -1588.05232204
  -1588.05232204 -1588.05232204]
 [ 5833.17626579  -152.33373421    39.67626579 ...  -166.82373421
   -166.82373421  -166.82373421]
 [14657.93471849 -1332.62528151 -1006.49528151 ... -1342.06528151
  -1342.06528151 -1342.06528151]]
Chosen random sample is No.0
interation (t): 1787
Chosen class is: purpose_car
[[ 6832.27198088  -359.53801912  -141.46801912 ...  -367.72801912
   -367.72801912  -367.72801912]
 [ 4838.51970107 -1145.46029893  -950.47029893 ... -1161.48029893
  -1161.48029893 -1161.48029893]
 [ 7349.12920591  -642.98079409  -400.58079409 ...  -650.8707

Chosen random sample is No.1475
interation (t): 1798
Chosen class is: purpose_educational
[[ 7.79215417e+03 -1.89175826e+02 -1.76582587e+00 ... -2.07845826e+02
  -2.07845826e+02 -2.07845826e+02]]
Chosen random sample is No.0
interation (t): 1799
Chosen class is: purpose_car
[[ 6830.23849598  -361.57150402  -143.50150402 ...  -369.76150402
   -369.76150402  -369.76150402]
 [ 4832.04241368 -1151.93758632  -956.94758632 ... -1167.95758632
  -1167.95758632 -1167.95758632]
 [ 7345.41859459  -646.69140541  -404.29140541 ...  -654.58140541
   -654.58140541  -654.58140541]
 ...
 [ 4023.15210305  -968.95789695  -820.41789695 ...  -976.84789695
   -976.84789695  -976.84789695]
 [ 4813.77006049  -176.29993951   -25.04993951 ...  -186.22993951
   -186.22993951  -186.22993951]
 [ 2517.73630634  -468.60369366  -380.22369366 ...  -482.26369366
   -482.26369366  -482.26369366]]
Chosen random sample is No.78
interation (t): 1800
Chosen class is: purpose_debt_consolidation
[[ 4.70683276e+03 -2.78697244e

interation (t): 1811
Chosen class is: purpose_educational
[[ 7.79102071e+03 -1.90309290e+02 -2.89929035e+00 ... -2.08979290e+02
  -2.08979290e+02 -2.08979290e+02]]
Chosen random sample is No.0
interation (t): 1812
Chosen class is: purpose_credit_card
[[30285.53702598 -4702.66297402 -3939.43297402 ... -4714.46297402
  -4714.46297402 -4714.46297402]
 [ 2660.00104093  -331.40895907  -245.16895907 ...  -339.99895907
   -339.99895907  -339.99895907]
 [ 7248.76740392  -138.49259608    97.17740392 ...  -151.23259608
   -151.23259608  -151.23259608]
 ...
 [19322.81851772  -667.10148228   -31.08148228 ...  -677.18148228
   -677.18148228  -677.18148228]
 [19216.65961978  -769.78038022  -322.52038022 ...  -783.34038022
   -783.34038022  -783.34038022]
 [ 8681.55142532  -556.45857468  -261.25857468 ...  -568.44857468
   -568.44857468  -568.44857468]]
Chosen random sample is No.1412
interation (t): 1813
Chosen class is: purpose_educational
[[ 7.79002068e+03 -1.91309318e+02 -3.89931842e+00 ... -2.09

In [89]:
dfSample2030 = dfS
dfSample2030.to_csv('sample2030.csv')